<a href="https://colab.research.google.com/github/nlquantumm-source/Cryo-CMOS-ICs/blob/main/Cryo_CMOS_ICs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Block 1: Imports and Fundamental Constants
# General Libraries
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider
import ipywidgets as widgets

# Physical Constants (using standard scipy.constants)
from scipy.constants import k, e, epsilon_0

# Semiconductor Parameters
EPS_SI = 11.7 * epsilon_0   # Permittivity of Silicon
EPS_OX = 3.9 * epsilon_0    # Permittivity of SiO2 (Oxide)
NI_300K = 1.5e10 * 1e6      # Intrinsic carriers at 300K (m^-3)
EG_0 = 1.12                 # Bandgap of Silicon (eV)

In [ ]:
# @title Block 2: Vth and SS Physics Models
def calculate_physics(temp, n_sub, t_ox, t_0):
    """
    Calculates Vth and SS based on Beckers/Bohuslavskyi models for Cryo-CMOS.

    Args:
        temp (float/array): Temperature in Kelvin.
        n_sub (float): Substrate Doping Concentration (cm^-3).
        t_ox (float): Oxide Thickness (nm).
        t_0 (float): Characteristic Disorder Temperature (K) for SS saturation.

    Returns:
        tuple: (v_th, ss, phi_f) - Threshold Voltage, Subthreshold Swing, Fermi Potential.
    """
    # Unit Conversions and Clamping
    T = np.maximum(temp, 1.0)  # Clamp T at 1K to avoid division by zero
    N_SUB = n_sub * 1e6        # Convert cm^-3 to m^-3
    C_OX = EPS_OX / (t_ox * 1e-9) # Oxide Capacitance (F/m^2)

    # -------------------------------------------------------
    # MODEL 1: Threshold Voltage (Vth) - Freeze-out Effect
    # -------------------------------------------------------
    # Intrinsic Carrier Scaling (n_i)
    exponent = -(EG_0 * e) / (2 * k * T)
    # Simplified approximation for n_i(T) relative to n_i(300K)
    n_i = NI_300K * (T / 300)**1.5 * np.exp(exponent + (EG_0 * e)/(2 * k * 300))
    n_i = np.maximum(n_i, 1e-30) # Prevent log(0)

    # Fermi Potential (Phi_F) - The main driver of Vth shift
    phi_f = (k * T / e) * np.log(N_SUB / n_i)

    # Threshold Voltage Calculation (Vth = V_FB + 2*Phi_F + Gamma * sqrt(2*Phi_F))
    v_fb = -0.9 # Flatband voltage (assumed)
    gamma = np.sqrt(2 * e * EPS_SI * N_SUB) / C_OX
    v_th = v_fb + 2 * phi_f + gamma * np.sqrt(2 * phi_f)

    # -------------------------------------------------------
    # MODEL 2: Subthreshold Swing (SS) - Saturation Effect
    # -------------------------------------------------------
    # Effective Temperature (T_eff) - Models saturation due to band tails/disorder
    t_eff = np.sqrt(T**2 + t_0**2)

    # SS Calculation (SS = ln(10) * k * T_eff / q * (1 + C_it/C_ox))
    # Assumes ideal interface (C_it=0) for simplicity
    ss = np.log(10) * (k * t_eff / e) * 1000 # Convert V to mV

    return v_th, ss, phi_f

In [ ]:
# @title Block 3: Interactive 2D Simulation (Vth & SS vs. Temperature)
def plot_simulation(n_doping_exp, t_ox_nm, t_0_k):
    """
    Plots the Vth and SS curves based on user slider inputs.
    """
    temp_range = np.linspace(4, 300, 100) # 4K to 300K
    n_doping = 10**n_doping_exp

    v_th, ss, _ = calculate_physics(temp_range, n_doping, t_ox_nm, t_0_k)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # --- Plot 1: Vth vs Temperature (Freeze-Out Penalty) ---
    ax1.plot(temp_range, v_th, 'b-', linewidth=2.5)
    ax1.set_title(f"Freeze-Out Penalty: Vth Shift (Doping=1e{n_doping_exp:.1f})")
    ax1.set_xlabel("Temperature (K)")
    ax1.set_ylabel("Threshold Voltage Vth (V)")
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.axvline(x=4, color='r', linestyle='--', label='4K Stage')
    ax1.legend()

    # --- Plot 2: SS vs Temperature (Saturation Effect) ---
    ax2.plot(temp_range, ss, 'r-', linewidth=2.5, label='Actual (Saturating)')
    # Plot ideal Boltzmann line for comparison
    ss_ideal = np.log(10) * (k * temp_range / e) * 1000
    ax2.plot(temp_range, ss_ideal, 'k:', label='Ideal (Boltzmann)')

    ax2.set_title(f"Cryo Limit: SS Saturation (T0 = {t_0_k}K)")
    ax2.set_xlabel("Temperature (K)")
    ax2.set_ylabel("Subthreshold Swing (mV/dec)")
    ax2.set_yscale('log')
    ax2.grid(True, which="both", linestyle='--', alpha=0.6)
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Execute the interactive simulation
print("Adjust the sliders below to see the Cryo-CMOS physics effects:")
interact(plot_simulation,
         n_doping_exp=FloatSlider(min=15, max=18, step=0.1, value=17, description='Doping (log10(cm^-3))'),
         t_ox_nm=FloatSlider(min=1, max=10, step=0.5, value=2, description='Oxide (nm)'),
         t_0_k=IntSlider(min=0, max=50, step=5, value=15, description='Disorder T0 (K)'));

## **Simulation Parameters & References**

### **1. Key Simulation Parameters**
| Parameter | Symbol | Range / Value | Description |
| :--- | :---: | :--- | :--- |
| **Temperature** | $T$ | $4\text{K} - 300\text{K}$ | Operating temperature range (Cryogenic to Room Temp). |
| **Substrate Doping** | $N_{sub}$ | $10^{15} - 10^{18} \text{ cm}^{-3}$ | P-type silicon doping concentration. |
| **Oxide Thickness** | $t_{ox}$ | $1 - 10 \text{ nm}$ | Gate oxide ($SiO_2$) thickness. |
| **Disorder Temp** | $T_0$ | $0 - 50 \text{K}$ | Characteristic temperature modeling interface disorder/band tails. |
| **Silicon Bandgap** | $E_{g0}$ | $1.12 \text{ eV}$ | Energy gap of Silicon at 0K (approx). |

### **2. Physics Models Used**

#### **A. Threshold Voltage ($V_{th}$) - Freeze-Out Effect**
*   **Mechanism:** At cryogenic temperatures, intrinsic carrier concentration ($n_i$) drops exponentially. The Fermi potential $\phi_F \propto T \cdot \ln(N_{sub}/n_i)$ increases, leading to a massive shift in $V_{th}$.
*   **Equation:** $V_{th} = V_{FB} + 2\phi_F + \gamma\sqrt{2\phi_F}$
*   **Approximation:** Simplified carrier freeze-out model assuming standard Boltzmann statistics (valid > 20K, approximated at 4K).

#### **B. Subthreshold Swing ($SS$) - Saturation**
*   **Mechanism:** $SS$ should theoretically go to 0 as $T \to 0$ ($SS = \ln(10)kT/q$). In reality, it saturates due to interface trap density and band-tail disorder.
*   **Model:** **Beckers/Bohuslavskyi Model** (Effective Temperature).
*   **Equation:** $SS = \ln(10) \frac{k \cdot T_{eff}}{q}$, where $T_{eff} = \sqrt{T^2 + T_0^2}$.

### **3. References**
These simulations are based on standard Cryo-CMOS compact modeling concepts found in:
*   *Beckers, I., et al.* "Characterization and modeling of 28 nm bulk CMOS technology down to 4.2 K." **Solid-State Electronics** (2018).
*   *Bohuslavskyi, H., et al.* "Cryogenic Characterization of 28nm FD-SOI." **Essderc** (2017).
*   *Ghibaudo, G.* "Low temperature characterization of MOSFETs." (Standard $T_{eff}$ methodology).

In [ ]:
# @title Block 4: 3D Visualization - The "Freeze-Out Mountain" (Animated)
import numpy as np
import plotly.graph_objects as go
from scipy.constants import k, e, epsilon_0

# --- Embedded Physics Model (Self-contained) ---
EPS_SI = 11.7 * epsilon_0
EPS_OX = 3.9 * epsilon_0
NI_300K = 1.5e10 * 1e6
EG_0 = 1.12

def calculate_physics(temp, n_sub, t_ox, t_0):
    T = np.maximum(temp, 1.0)
    N_SUB = n_sub * 1e6
    C_OX = EPS_OX / (t_ox * 1e-9)

    exponent = -(EG_0 * e) / (2 * k * T)
    n_i = NI_300K * (T / 300)**1.5 * np.exp(exponent + (EG_0 * e)/(2 * k * 300))
    n_i = np.maximum(n_i, 1e-30)

    phi_f = (k * T / e) * np.log(N_SUB / n_i)
    v_fb = -0.9
    gamma = np.sqrt(2 * e * EPS_SI * N_SUB) / C_OX
    v_th = v_fb + 2 * phi_f + gamma * np.sqrt(2 * phi_f)

    t_eff = np.sqrt(T**2 + t_0**2)
    ss = np.log(10) * (k * t_eff / e) * 1000
    return v_th, ss, phi_f

# --- Animation Code ---
def plot_3d_surface_animated():
    """
    Generates an animated 3D surface plot of Vth as a function of Temperature and Doping.
    Includes a 'Play' button to rotate the camera.
    """
    # Create Meshgrid for variables
    temp_vals = np.linspace(4, 300, 50)
    doping_exps = np.linspace(15, 18, 50)
    T_mesh, D_mesh = np.meshgrid(temp_vals, doping_exps)
    N_mesh = 10**D_mesh

    # Calculate Vth for every point
    v_th_surface, _, _ = calculate_physics(T_mesh, N_mesh, t_ox=2.0, t_0=15)

    # --- Animation Setup ---
    frames = []
    n_frames = 60  # Number of frames for a full rotation
    for k in range(n_frames):
        angle = 2 * np.pi * k / n_frames
        x_eye = 2.0 * np.cos(angle)
        y_eye = 2.0 * np.sin(angle)

        frames.append(go.Frame(
            layout=dict(scene=dict(camera=dict(eye=dict(x=x_eye, y=y_eye, z=1.25))))
        ))

    # Create 3D Plot using Plotly
    fig = go.Figure(
        data=[go.Surface(z=v_th_surface, x=T_mesh, y=D_mesh, colorscale='Viridis')],
        frames=frames
    )

    fig.update_layout(
        title='3D Cryo-CMOS Model: Vth Dependence (Click Play to Rotate)',
        scene = dict(
            xaxis_title='Temperature (K)',
            yaxis_title='Doping (log 10^x cm^-3)',
            zaxis_title='Threshold Voltage (V)',
            camera=dict(eye=dict(x=2.0, y=0, z=1.25))
        ),
        width=800, height=600,
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            y=1, x=0.8, xanchor='left', yanchor='bottom',
            pad=dict(t=45, r=10),
            buttons=[dict(
                label='▶ Play Rotation',
                method='animate',
                args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True, mode='immediate')]
            )]
        )]
    )
    fig.show()

plot_3d_surface_animated()

In [ ]:
# @title Block 5: 3D Visualization of SS Saturation (Animated)
import numpy as np
import plotly.graph_objects as go
from scipy.constants import k, e, epsilon_0

# --- Embedded Physics Model (Self-contained) ---
EPS_SI = 11.7 * epsilon_0
EPS_OX = 3.9 * epsilon_0
NI_300K = 1.5e10 * 1e6
EG_0 = 1.12

def calculate_physics(temp, n_sub, t_ox, t_0):
    T = np.maximum(temp, 1.0)
    N_SUB = n_sub * 1e6
    C_OX = EPS_OX / (t_ox * 1e-9)

    exponent = -(EG_0 * e) / (2 * k * T)
    n_i = NI_300K * (T / 300)**1.5 * np.exp(exponent + (EG_0 * e)/(2 * k * 300))
    n_i = np.maximum(n_i, 1e-30)

    phi_f = (k * T / e) * np.log(N_SUB / n_i)
    v_fb = -0.9
    gamma = np.sqrt(2 * e * EPS_SI * N_SUB) / C_OX
    v_th = v_fb + 2 * phi_f + gamma * np.sqrt(2 * phi_f)

    t_eff = np.sqrt(T**2 + t_0**2)
    ss = np.log(10) * (k * t_eff / e) * 1000
    return v_th, ss, phi_f

# --- Animation Code ---
def plot_3d_ss_saturation_animated():
    """
    Generates an animated 3D surface plot of SS as a function of Temperature and T0.
    Includes a 'Play' button to rotate the camera.
    """
    # Create Meshgrid for variables
    temp_vals = np.linspace(4, 300, 50)
    t0_vals = np.linspace(0, 50, 50)
    T_mesh, T0_mesh = np.meshgrid(temp_vals, t0_vals)

    # Calculate SS
    _, ss_surface, _ = calculate_physics(temp=T_mesh, n_sub=1e17, t_ox=2.0, t_0=T0_mesh)

    # --- Animation Setup ---
    frames = []
    n_frames = 60
    for k in range(n_frames):
        angle = 2 * np.pi * k / n_frames
        x_eye = 2.2 * np.cos(angle)
        y_eye = 2.2 * np.sin(angle)

        frames.append(go.Frame(
            layout=dict(scene=dict(camera=dict(eye=dict(x=x_eye, y=y_eye, z=1.4))))
        ))

    # Create 3D Plot using Plotly
    fig = go.Figure(
        data=[go.Surface(z=ss_surface, x=T_mesh, y=T0_mesh, colorscale='Plasma')],
        frames=frames
    )

    fig.update_layout(
        title='3D Cryo-CMOS Model: SS Saturation Floor (Click Play to Rotate)',
        scene = dict(
            xaxis_title='Physical Temperature T (K)',
            yaxis_title='Disorder Characteristic Temp T0 (K)',
            zaxis_title='Subthreshold Swing SS (mV/dec)',
            camera=dict(eye=dict(x=2.2, y=0, z=1.4))
        ),
        width=800, height=600,
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            y=1, x=0.8, xanchor='left', yanchor='bottom',
            pad=dict(t=45, r=10),
            buttons=[dict(
                label='▶ Play Rotation',
                method='animate',
                args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True, mode='immediate')]
            )]
        )]
    )
    fig.show()

plot_3d_ss_saturation_animated()

In [ ]:
# @title Block 6a: Install Video Export Dependencies
# Install specific version of kaleido with bundled engine, and latest moviepy/plotly
!pip install "kaleido==0.2.1" "moviepy>=2.0.0" plotly
print("Dependencies installed. Please run the next cell to generate videos.")

In [ ]:
# @title Block 6b: Generate and Download MP4 Videos & Static Plot
# --- Auto-Install Dependencies ---
!pip install "kaleido==0.2.1" "moviepy>=2.0.0" plotly

import sys
import os
import shutil
import warnings
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import k, e, epsilon_0
from google.colab import files

# --- FIX: Aggressive Reload of Plotly/Kaleido ---
# Remove modules from memory to force a fresh re-import
# This fixes the issue where Plotly doesn't see Kaleido if installed in the same session.
modules_to_reload = [m for m in sys.modules.keys() if "plotly" in m or "kaleido" in m]
for m in modules_to_reload:
    del sys.modules[m]

# Now Import Plotly fresh
import plotly.graph_objects as go
import plotly.io as pio

# --- Import MoviePy ---
try:
    from moviepy import ImageSequenceClip
except ImportError:
    try:
        from moviepy.editor import ImageSequenceClip
    except ImportError:
        print("Error: Could not import ImageSequenceClip from moviepy.")

# Suppress syntax warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

# --- Physics Constants & Model (Self-Contained) ---
EPS_SI = 11.7 * epsilon_0
EPS_OX = 3.9 * epsilon_0
NI_300K = 1.5e10 * 1e6
EG_0 = 1.12

def calculate_physics_export(temp, n_sub, t_ox, t_0):
    T = np.maximum(temp, 1.0)
    N_SUB = n_sub * 1e6
    C_OX = EPS_OX / (t_ox * 1e-9)
    exponent = -(EG_0 * e) / (2 * k * T)
    n_i = NI_300K * (T / 300)**1.5 * np.exp(exponent + (EG_0 * e)/(2 * k * 300))
    n_i = np.maximum(n_i, 1e-30)
    phi_f = (k * T / e) * np.log(N_SUB / n_i)
    v_fb = -0.9
    gamma = np.sqrt(2 * e * EPS_SI * N_SUB) / C_OX
    v_th = v_fb + 2 * phi_f + gamma * np.sqrt(2 * phi_f)
    t_eff = np.sqrt(T**2 + t_0**2)
    ss = np.log(10) * (k * t_eff / e) * 1000
    return v_th, ss

# --- PART 1: Download Static Image (From Block 3) ---
def download_static_plot_block3():
    print("Generating static plot from Block 3 parameters...")
    # Default Parameters from Block 3
    n_doping_exp = 17.0
    t_ox_nm = 2.0
    t_0_k = 15

    temp_range = np.linspace(4, 300, 100)
    n_doping = 10**n_doping_exp
    v_th, ss = calculate_physics_export(temp_range, n_doping, t_ox_nm, t_0_k)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot 1: Vth
    ax1.plot(temp_range, v_th, 'b-', linewidth=2.5)
    ax1.set_title(f"Freeze-Out Penalty: Vth Shift (Doping=1e{n_doping_exp:.1f})")
    ax1.set_xlabel("Temperature (K)")
    ax1.set_ylabel("Threshold Voltage Vth (V)")
    ax1.grid(True, linestyle='--', alpha=0.6)

    # Plot 2: SS
    ax2.plot(temp_range, ss, 'r-', linewidth=2.5, label='Actual (Saturating)')
    ss_ideal = np.log(10) * (k * temp_range / e) * 1000
    ax2.plot(temp_range, ss_ideal, 'k:', label='Ideal (Boltzmann)')
    ax2.set_title(f"Cryo Limit: SS Saturation (T0 = {t_0_k}K)")
    ax2.set_xlabel("Temperature (K)")
    ax2.set_ylabel("Subthreshold Swing (mV/dec)")
    ax2.set_yscale('log')
    ax2.grid(True, which="both", linestyle='--', alpha=0.6)
    ax2.legend()

    plt.tight_layout()
    filename = "Cryo_CMOS_Block3_Plot.png"
    plt.savefig(filename, dpi=300)
    plt.close(fig)

    files.download(filename)
    print(f"Downloaded {filename}\n")

# Execute Image Download
download_static_plot_block3()

# --- PART 2: Generate MP4 Videos (3D) ---
def make_video(filename, title, z_axis, eye_r=2.0, z_eye=1.25):
    print(f"Processing {filename}...")
    # Generate Data
    temp_vals = np.linspace(4, 300, 50)
    if "Vth" in filename:
        doping_exps = np.linspace(15, 18, 50)
        X, Y = np.meshgrid(temp_vals, doping_exps)
        x_label, y_label = 'Temperature (K)', 'Doping'
        Z, _ = calculate_physics_export(X, 10**Y, 2.0, 15)
        colorscale = 'Viridis'
    else:
        t0_vals = np.linspace(0, 50, 50)
        X, Y = np.meshgrid(temp_vals, t0_vals)
        x_label, y_label = 'Temperature (K)', 'Disorder T0 (K)'
        _, Z = calculate_physics_export(X, 1e17, 2.0, Y)
        colorscale = 'Plasma'

    # Prepare Directory
    frame_dir = "frames_temp"
    if os.path.exists(frame_dir):
        shutil.rmtree(frame_dir)
    os.makedirs(frame_dir)

    # Base Figure
    fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale=colorscale)])
    fig.update_layout(
        title=title,
        scene=dict(xaxis_title=x_label, yaxis_title=y_label, zaxis_title=z_axis),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    # Render Frames
    print("Rendering 60 frames...")
    frames = []
    n_frames = 60

    try:
        for i in range(n_frames):
            angle = 2 * np.pi * i / n_frames
            x_eye = eye_r * np.cos(angle)
            y_eye = eye_r * np.sin(angle)
            fig.update_layout(scene_camera=dict(eye=dict(x=x_eye, y=y_eye, z=z_eye)))

            frame_path = os.path.join(frame_dir, f"frame_{i:03d}.png")
            # Use Kaleido engine explicitly or let plotly decide (Kaleido is default for static export)
            fig.write_image(frame_path, width=800, height=600, scale=1.5)
            frames.append(frame_path)

        print(f"Stitching {filename}...")
        clip = ImageSequenceClip(frames, fps=15)
        clip.write_videofile(filename, codec='libx264', audio=False, logger=None)

        shutil.rmtree(frame_dir)
        print(f"Done: {filename}")
        files.download(filename)

    except Exception as e:
        print(f"\nError processing {filename}: {str(e)}")
        if "kaleido" in str(e).lower():
            print("Tip: If Kaleido issues persist, try restarting the runtime (Runtime -> Restart Session) and running this cell again.")
        raise e

# Execute Video Generation
make_video("Cryo_CMOS_Vth.mp4", "Vth Freeze-Out Model", "Vth (V)", eye_r=2.0, z_eye=1.25)
make_video("Cryo_CMOS_SS.mp4", "SS Saturation Model", "SS (mV/dec)", eye_r=2.2, z_eye=1.4)